In [ ]:
from sage.all import (QQ, sigma, nth_prime, matrix, flatten, polygen)
import pickle

Rc.<c> = QQ[]
Rn.<n> = QQ[]
Rd.<d>=QQ[]
Rkk.<kk>=QQ[]

def tau(n):
    def step1(n, k):
        return 35*k^4 - 52*k^3*n + 18*k^2*n^2

    def step2(n, k):
        return sigma(k) * sigma(n - k)

    def step3(n):
        ans = 0
        for k in range(1, n):
            ans += step1(n, k) * step2(n, k)
        return 24*ans

    return n^4 * sigma(n) - step3(n)


def tau_prime(k):
    return tau(nth_prime(k))

def P_matrix(lst):
    L = len(lst)
    columns = [lst]
    for k in range(1, L):
        column = [0]*(k-1) + [-k]
        column = flatten(column + [lst])
        column = column + [lst]
        column = column[:L]
        columns.append(column)
    return matrix(columns).transpose()


            
tau_prime_list=[(k,tau_prime(k)) for k in range(1,40)]
h_list=[[0,1]]+tau_prime_list  
c_polys=[]
c_cffs=[]

for n in range(2,10):
    print("n:",n)
    data=[]
    
    for c in range (1,n+5):
        j_c_list=[(0,c)]
        
        for k in range(1,n+1):
            sm=0
            
            for r in range(1,k):
                sm+=j_c_list[r][1]*h_list[k-r][1] 
    
            j_k=k*h_list[k][1]-sm
            j_c_list.append((k,j_k))
    
        sliced = j_c_list[:n]
        inputs = [pair[1] for pair in sliced]
        det = P_matrix(inputs).det()
        data.append((c, det))
        
    c_poly = Rc.lagrange_polynomial(data)
    

    c_polys.append((n, c_poly))
    
for pair in c_polys:
    print("n:",pair[0])
    print(pair[1])
    print()    


In [24]:
import pickle
from sage.all import QQ, sigma, nth_prime, matrix, flatten, polygen, RR, CC, factorial
import ast
import math
from scipy.signal import find_peaks, correlate
from scipy.fft import fft, fftfreq
import numpy as np

Rc.<c> = QQ[] 

def tau(n):
    def step1(n, k):
        return 35*k**4 - 52*k**3*n + 18*k**2*n**2
    def step2(n, k):
        return sigma(k) * sigma(n - k)
    def step3(n):
        ans = 0
        for k in range(1, n):
            ans = ans + step1(n, k) * step2(n, k)
        return 24*ans
    return n**4 * sigma(n) - step3(n)

def tau_prime(k):
    pn = nth_prime(k)
    return tau(pn)

def P_matrix(lst):
    lenlist = len(lst)
    columns = [lst]
    for k in range(1, lenlist):
        column = [0]*(k-1)
        column = column + [-k]
        column = flatten(column + [lst])
        column = column + [lst]
        column = column[:lenlist]
        columns += [column]
    return (matrix(columns)).transpose()

precision_bits = 100
x = polygen(QQ)

tau_prime_list = [(k, tau_prime(k)) for k in range(1, 40)]
h_list = [(0, 1)] + tau_prime_list  # h_list[k] = (k, h[k]) where h[0]=1, h[k]=tau(p_k)
c_polys = []

for n in range(2, 10):
    print("n:", n)
    
    points_for_interpolation = []
    
    for c_val in range(1, n+5):
        # Build j_c_list to match the structure of the correct cell
        j_c_list = [(0, c_val)]  # j_c_list[0] = (0, c_val), representing j[1] = c_val
        
        for k in range(1, n+1):
            sm = 0
            for r in range(1, k):
                sm += j_c_list[r][1] * h_list[k-r][1]
            j_k = k * h_list[k][1] - sm
            j_c_list.append((k, j_k))
        
        # Now compute h_c using equation D1
        # h_c[0] = 1, and h_c[k] = (1/k) * Σ(r=1 to k) j_c[r] * h_c[k-r]
        # But j_c_list[r] corresponds to j[r+1], so we need to adjust indices
        
        # Actually, let me extract j values properly:
        # j_c_list[0][1] is j[1], j_c_list[1][1] is j[2], etc.
        j_values = [0] + [j_c_list[i][1] for i in range(len(j_c_list))]  # j_values[k] = j[k]
        
        h_c = [1]  # h_c[0] = 1
        for k in range(1, n+1):
            sm = 0
            for r in range(1, k+1):
                sm += j_values[r] * h_c[k-r]
            h_c.append(sm / k)
        
        points_for_interpolation.append((c_val, h_c[n] * factorial(n)))
    
    c_poly = Rc.lagrange_polynomial(points_for_interpolation)
    c_polys.append((n, c_poly))

for pair in c_polys:
    print("n:", pair[0])
    print(pair[1])
    print()

n: 2
n: 3
n: 4
n: 5
n: 6
n: 7
n: 8
n: 9
n: 2
c^2 - 24

n: 3
c^3 - 72*c - 144

n: 4
c^4 - 144*c^2 - 576*c + 114588

n: 5
c^5 - 240*c^3 - 1440*c^2 + 572940*c + 12479232

n: 6
c^6 - 360*c^4 - 2880*c^3 + 1718820*c^2 + 74875392*c + 1198192320

n: 7
c^7 - 504*c^5 - 5040*c^4 + 4010580*c^3 + 262063872*c^2 + 8387346240*c + 17919600192

n: 8
c^8 - 672*c^6 - 8064*c^5 + 8021160*c^4 + 698836992*c^3 + 33549384960*c^2 + 143356801536*c - 20578729954608

n: 9
c^9 - 864*c^7 - 12096*c^6 + 14438088*c^5 + 1572383232*c^4 + 100648154880*c^3 + 645105606912*c^2 - 185208569591472*c - 6136737862569984



In [25]:
import pickle
from sage.all import QQ, sigma, nth_prime, matrix, flatten, polygen, RR, CC, factorial
import ast
import math
from scipy.signal import find_peaks, correlate
from scipy.fft import fft, fftfreq
import numpy as np

Rc.<c> = QQ[] 

def tau(n):
    def step1(n, k):
        return 35*k**4 - 52*k**3*n + 18*k**2*n**2
    def step2(n, k):
        return sigma(k) * sigma(n - k)
    def step3(n):
        ans = 0
        for k in range(1, n):
            ans = ans + step1(n, k) * step2(n, k)
        return 24*ans
    return n**4 * sigma(n) - step3(n)

def tau_prime(k):
    pn = nth_prime(k)
    return tau(pn)

def P_matrix(lst):
    lenlist = len(lst)
    columns = [lst]
    for k in range(1, lenlist):
        column = [0]*(k-1)
        column = column + [-k]
        column = flatten(column + [lst])
        column = column + [lst]
        column = column[:lenlist]
        columns += [column]
    return (matrix(columns)).transpose()

precision_bits = 100
x = polygen(QQ)

tau_prime_list = [(k, tau_prime(k)) for k in range(1, 401)]
h_list = [(0, 1)] + tau_prime_list  # h_list[k] = (k, h[k]) where h[0]=1, h[k]=tau(p_k)
c_polys = []

for n in range(2, 400):
    print("n:", n)
    
    points_for_interpolation = []
    
    for c_val in range(1, n+5):
        # Build j_c_list to match the structure of the correct cell
        j_c_list = [(0, c_val)]  # j_c_list[0] = (0, c_val), representing j[1] = c_val
        
        for k in range(1, n+1):
            sm = 0
            for r in range(1, k):
                sm += j_c_list[r][1] * h_list[k-r][1]
            j_k = k * h_list[k][1] - sm
            j_c_list.append((k, j_k))
        
        # Now compute h_c using equation D1
        # h_c[0] = 1, and h_c[k] = (1/k) * Σ(r=1 to k) j_c[r] * h_c[k-r]
        # But j_c_list[r] corresponds to j[r+1], so we need to adjust indices
        
        # Actually, let me extract j values properly:
        # j_c_list[0][1] is j[1], j_c_list[1][1] is j[2], etc.
        j_values = [0] + [j_c_list[i][1] for i in range(len(j_c_list))]  # j_values[k] = j[k]
        
        h_c = [1]  # h_c[0] = 1
        for k in range(1, n+1):
            sm = 0
            for r in range(1, k+1):
                sm += j_values[r] * h_c[k-r]
            h_c.append(sm / k)
        
        points_for_interpolation.append((c_val, h_c[n] * factorial(n)))
    
    c_poly = Rc.lagrange_polynomial(points_for_interpolation)
    c_polys.append((n, c_poly))

for pair in c_polys[:5]:
    print("n:", pair[0])
    print(pair[1])
    print()

n: 2
n: 3
n: 4
n: 5
n: 6
n: 7
n: 8
n: 9
n: 2
c^2 - 24

n: 3
c^3 - 72*c - 144

n: 4
c^4 - 144*c^2 - 576*c + 114588

n: 5
c^5 - 240*c^3 - 1440*c^2 + 572940*c + 12479232

n: 6
c^6 - 360*c^4 - 2880*c^3 + 1718820*c^2 + 74875392*c + 1198192320

